In [ ]:
# duplicar as últimas amostras

In [1]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

%load_ext autoreload

sys.path.append(os.getcwd()+'//..//scripts//')
path = os.getcwd() + '//..//'

from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGFeaturesSize import MNGFeaturesSize
from MNGModel import MNGModel

In [2]:
# repetindo os últimos dados

In [2]:
data_path = os.getcwd()+'/../resampling/all_data_repeated.csv'
# data_path = os.getcwd()+'/../resampling/all_data_synthetic.csv'
old_data = pd.read_csv(data_path, sep=',', index_col=0)

In [4]:
# run this once
# repeated_list_indexes = np.arange(480)
# random.shuffle(repeated_list_indexes)
# to_be_deleted = repeated_list_indexes[:120].tolist()

In [3]:
to_be_deleted = [234, 385, 248, 110, 378, 159, 131, 252, 9, 53, 46, 153, 307, 203, 414, 412, 438, 20, 208, 188, \
                 357, 315, 246, 219, 235, 456, 261, 51, 339, 415, 67, 373, 52, 335, 201, 95, 129, 200, 364, 81, \
                 172, 291, 64, 367, 71, 255, 263, 40, 268, 165, 313, 316, 416, 404, 167, 238, 475, 32, 202, 345, \
                 294, 139, 324, 83, 15, 86, 254, 419, 477, 447, 207, 94, 318, 329, 249, 250, 85, 186, 361, 170, \
                 270, 100, 421, 258, 96, 69, 397, 451, 22, 181, 17, 59, 467, 215, 128, 230, 93, 286, 461, 337, \
                 468, 18, 271, 214, 391, 383, 82, 325, 228, 442, 274, 231, 122, 426, 401, 141, 143, 244, 192, 77]

In [4]:
# ATTENTION! I'M ASSIGNING IT MANUALLY, NOT BASED IN INDEX
resampled_data = old_data.iloc[-480:]
rows_to_delete = resampled_data.iloc[to_be_deleted]

data = old_data.drop(rows_to_delete.index.values)

# remove atributos diff
data = data.drop(columns=data.columns.values[:1710])

In [5]:
n_var_names = list(filter(lambda x: 'region' in x, data.columns.values))

In [6]:
data = data.drop(columns=n_var_names)

In [109]:
# data = pd.read_csv(path+'features//size_all_half.csv', sep=';', index_col=0)
# palmer_sst = pd.read_csv(path+'//reference_values//primeiros_dados_palmer_sst.csv', sep=';', index_col=0)
# palmer_firm = pd.read_csv(path+'//reference_values//primeiros_dados_palmer_firmeza.csv', sep=';', index_col=0)

# data['sst'] = palmer_sst.values
# data['firmeza'] = palmer_firm.values

In [7]:
X = data.drop(columns=['time', 'sst', 'firmeza', 'ratio', 'massa', 'acidez'])
Y_sst = data['sst']
Y_firm = data['firmeza']
Y_time = data['time']
Y_acidez = data['acidez']
Y_massa = data['massa']
Y_ratio = data['ratio']

In [8]:
len(data.drop(columns=['width', 'height', 'time', 'sst', 'firmeza', 'ratio', 'acidez']).columns)

41

In [9]:
predictions = pd.DataFrame(index=data.index.values, columns=['sst_predict', 'firmeza_predict', 'time_predict', \
                                                             'massa_predict', 'ratio_predict', 'acidez_predict'])
real = pd.DataFrame(index=data.index.values, columns=['sst_real', 'firmeza_real', 'time_real', 'massa_real', \
                                                      'ratio_real', 'acidez_real'])

# execute the following 2 cells to run models without resampled data

In [50]:
synthetic_indexes = list(filter(lambda x: 'synthetic' in x, X.index.values))
rows_to_delete = X.loc[synthetic_indexes]
X = X.drop(rows_to_delete.index.values)
Y_sst = Y_sst.drop(rows_to_delete.index.values)
Y_firm = Y_firm.drop(rows_to_delete.index.values)
Y_time = Y_time.drop(rows_to_delete.index.values)
Y_acidez = Y_acidez.drop(rows_to_delete.index.values)
Y_massa = Y_massa.drop(rows_to_delete.index.values)
Y_ratio = Y_ratio.drop(rows_to_delete.index.values)

In [81]:
# execute this to run models without resampled data
predictions = pd.DataFrame(index=X.index.values, columns=['sst_predict', 'firmeza_predict', 'time_predict', \
                                                             'massa_predict', 'ratio_predict', 'acidez_predict'])
real = pd.DataFrame(index=X.index.values, columns=['sst_real', 'firmeza_real', 'time_real', 'massa_real', \
                                                      'ratio_real', 'acidez_real'])

In [12]:
n_trees = 100
kf = KFold(n_splits=5, shuffle=True)

In [31]:
def mlr_exp():
	i=1
	for train_i, test_i in kf.split(X):
		print(i)
		X_train, X_test = X.iloc[train_i], X.iloc[test_i]
		Y_sst_train, __ = Y_sst.iloc[train_i], Y_sst.iloc[test_i]
		Y_firm_train, __ = Y_firm.iloc[train_i], Y_firm.iloc[test_i]
		Y_time_train, __ = Y_time.iloc[train_i], Y_time.iloc[test_i]
		Y_acidez_train, __ = Y_acidez.iloc[train_i], Y_acidez.iloc[test_i]
		Y_massa_train, __ = Y_massa.iloc[train_i], Y_massa.iloc[test_i]
		Y_ratio_train, __ = Y_ratio.iloc[train_i], Y_ratio.iloc[test_i]
		
		print('sst')
		mlr_model = LinearRegression().fit(X_train, Y_sst_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['sst_predict'].iloc[test_i] = Y_predicted
		
		print('firmeza')
		mlr_model = LinearRegression().fit(X_train, Y_firm_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['firmeza_predict'].iloc[test_i] = Y_predicted
		
		print('time')
		mlr_model = LinearRegression().fit(X_train, Y_time_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['time_predict'].iloc[test_i] = Y_predicted

		print('acidez')
		mlr_model = LinearRegression().fit(X_train, Y_acidez_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['acidez_predict'].iloc[test_i] = Y_predicted

		print('massa')
		mlr_model = LinearRegression().fit(X_train, Y_massa_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['massa_predict'].iloc[test_i] = Y_predicted

		print('ratio')
		mlr_model = LinearRegression().fit(X_train, Y_ratio_train)
		Y_predicted = mlr_model.predict(X_test)
		predictions['ratio_predict'].iloc[test_i] = Y_predicted

		i=i+1	

In [32]:
mlr_exp()

1
sst
firmeza
time
acidez
massa
ratio
2
sst
firmeza
time
acidez
massa
ratio
3
sst
firmeza
time
acidez
massa
ratio
4
sst
firmeza
time
acidez
massa
ratio
5
sst
firmeza
time
acidez
massa
ratio


In [14]:
def rf_exp():
    i=1
    for train_i, test_i in kf.split(X):
        print(i)
        X_train, X_test = X.iloc[train_i], X.iloc[test_i]
        Y_sst_train, __ = Y_sst.iloc[train_i], Y_sst.iloc[test_i]
        Y_firm_train, __ = Y_firm.iloc[train_i], Y_firm.iloc[test_i]
        Y_time_train, __ = Y_time.iloc[train_i], Y_time.iloc[test_i]
        Y_acidez_train, __ = Y_acidez.iloc[train_i], Y_acidez.iloc[test_i]
        Y_massa_train, __ = Y_massa.iloc[train_i], Y_massa.iloc[test_i]
        Y_ratio_train, __ = Y_ratio.iloc[train_i], Y_ratio.iloc[test_i]

        print('sst')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_sst_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['sst_predict'].iloc[test_i] = Y_predicted

        print('firmeza')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_firm_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['firmeza_predict'].iloc[test_i] = Y_predicted

        print('time')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_time_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['time_predict'].iloc[test_i] = Y_predicted

        print('acidez')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_acidez_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['acidez_predict'].iloc[test_i] = Y_predicted

        print('massa')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_massa_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['massa_predict'].iloc[test_i] = Y_predicted

        print('ratio')
        rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_ratio_train)
        Y_predicted = rf_model.predict(X_test)
        predictions['ratio_predict'].iloc[test_i] = Y_predicted

        i=i+1    

In [15]:
rf_exp()

1
sst
firmeza
time
acidez
massa
ratio
2
sst
firmeza
time
acidez
massa
ratio
3
sst
firmeza
time
acidez
massa
ratio
4
sst
firmeza
time
acidez
massa
ratio
5
sst
firmeza
time
acidez
massa
ratio


# execute this to run models without resampled data

In [144]:
real['sst_real'] = data['sst'].drop(rows_to_delete.index.values)
real['firmeza_real'] = data['firmeza'].drop(rows_to_delete.index.values)
real['time_real'] = data['time'].drop(rows_to_delete.index.values)
real['massa_real'] = data['massa'].drop(rows_to_delete.index.values)
real['ratio_real'] = data['ratio'].drop(rows_to_delete.index.values)
real['acidez_real'] = data['acidez'].drop(rows_to_delete.index.values)

# real.iloc[660:] = real.iloc[-120:-60].values
# real.to_csv('real_repeated.csv', sep=',')

KeyError: "['1885_repeated' '2036_repeated' '1899_repeated' '1761_repeated'\n '2029_repeated' '1810_repeated' '1782_repeated' '1903_repeated'\n '1660_repeated' '1704_repeated' '1697_repeated' '1804_repeated'\n '1958_repeated' '1854_repeated' '2065_repeated' '2063_repeated'\n '2089_repeated' '1671_repeated' '1859_repeated' '1839_repeated'\n '2008_repeated' '1966_repeated' '1897_repeated' '1870_repeated'\n '1886_repeated' '2107_repeated' '1912_repeated' '1702_repeated'\n '1990_repeated' '2066_repeated' '1718_repeated' '2024_repeated'\n '1703_repeated' '1986_repeated' '1852_repeated' '1746_repeated'\n '1780_repeated' '1851_repeated' '2015_repeated' '1732_repeated'\n '1823_repeated' '1942_repeated' '1715_repeated' '2018_repeated'\n '1722_repeated' '1906_repeated' '1914_repeated' '1691_repeated'\n '1919_repeated' '1816_repeated' '1964_repeated' '1967_repeated'\n '2067_repeated' '2055_repeated' '1818_repeated' '1889_repeated'\n '2126_repeated' '1683_repeated' '1853_repeated' '1996_repeated'\

In [17]:
real['sst_real'] = data['sst']
real['firmeza_real'] = data['firmeza']
real['time_real'] = data['time']
real['massa_real'] = data['massa']
real['ratio_real'] = data['ratio']
real['acidez_real'] = data['acidez']

# real.iloc[660:] = real.iloc[-120:-60].values
# real.to_csv('real_repeated.csv', sep=',')

In [20]:
predictions['sst_real'] = real['sst_real']
predictions['firmeza_real'] = real['firmeza_real']
predictions['time_real'] = real['time_real']
predictions['acidez_real'] = real['acidez_real']
predictions['massa_real'] = real['massa_real']
predictions['ratio_real'] = real['ratio_real']

predictions.to_csv('../predictions/dados_predict_reais_all.csv', sep=';')

In [135]:
def calc_tabela(data, att, range_max, range_min, intervals, consider_end):
    indexes = []
    bins = math.floor((range_max-range_min)/intervals)
    tabela = pd.DataFrame(data=0,index=np.arange(bins),columns=data.columns)
    
    for bin_num in range(bins):
        min_v = range_min if bin_num == 0 else range_min + intervals*bin_num + 1
        if bin_num!=(bins-1):
            max_v = min_v + intervals if bin_num == 0 else min_v + intervals - 1
        else:
            max_v = range_max
        indexes.append(str(min_v)+'-'+str(max_v))
        tabela.iloc[bin_num] = data[(min_v<=data[att]) & (data[att]<=max_v)].mean()
    
    tabela.index = indexes
    return tabela

In [146]:
predictions = pd.read_csv('../predictions/dados_predict_reais(all-novo_repetição).csv', index_col=0, sep=';')

In [147]:
range_max = 170
range_min = 20
intervals = 15

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, False)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, False)

In [148]:
tabela_pred[tabela_pred.columns[:6]]

,sst_predict,firmeza_predict,time_predict,massa_predict,ratio_predict,acidez_predict
20-35,4.829745,93.665078,35.000000,62.789269,3.786356,1.620933
36-50,5.085197,114.444648,48.978689,174.760120,3.356270,1.796767
51-65,5.274651,123.906244,64.344186,294.305691,3.559763,1.521507
66-80,5.465300,125.690775,74.900000,357.470445,3.869573,1.466919
81-95,5.658827,133.727712,88.617308,466.071998,6.397387,1.396421
96-110,5.650183,142.214401,103.123239,514.281137,5.370420,1.124181
111-125,6.040062,137.969313,117.750000,550.008842,13.776546,0.951189
126-140,6.169157,140.630164,133.714286,535.795735,9.239433,0.810614
141-155,6.410046,137.821838,148.447685,579.691010,11.542422,0.685654
156-170,7.231733,119.231133,160.254444,581.637208,22.597702,0.627323


In [149]:
tabela_real

,sst_real,firmeza_real,time_real,massa_real,ratio_real,acidez_real
20-35,4.756667,93.146667,35.0,63.092500,3.247261,1.586642
36-50,5.081667,116.263333,50.0,187.278667,2.751712,1.856444
51-65,5.246667,123.836667,65.0,299.530333,3.604593,1.471272
66-80,5.795000,126.481667,80.0,438.429500,3.816705,1.533101
81-95,5.440000,145.780000,95.0,503.413667,4.636882,1.348623
96-110,5.770000,145.146667,110.0,520.198333,6.164416,0.960256
111-125,5.848333,148.991667,125.0,536.566833,7.082080,0.853902
126-140,5.973333,138.843333,140.0,573.752667,9.430505,0.662402
141-155,6.480000,140.870833,155.0,591.563667,10.655754,0.648803
156-170,6.810000,128.510833,170.0,586.664767,10.459548,0.673845


In [272]:
range_max = 190
range_min = 171
intervals = 9

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, True)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, True)

In [273]:
tabela_pred[tabela_pred.columns[:6]]

,sst_predict,firmeza_predict,time_predict,massa_predict,ratio_predict,acidez_predict
171-180,12.002298,39.908426,179.290426,463.112761,40.824573,0.505256
181-190,14.958847,7.216390,188.499568,425.672305,214.701453,0.087329


In [162]:
predictions['time_predict'].to_csv('means_size.csv')